In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import tensorflow as tf
import tensorflow.keras.datasets as ds

In [ ]:
print(tf.__version__)

# MNIST 학습데이터 준비

In [ ]:
# MNIST
(x_train,y_train),(x_test,y_test) = ds.mnist.load_data()

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
# 학습 데이터 정규화
x_trainF = x_train.reshape(60000,784)   # 1차원 구조로 변경
x_testF = x_test.reshape(10000,784)

x_trainF = x_trainF.astype('float32')   # 0~1로 정규화하기 위해 실수형으로 변환
x_testF = x_testF.astype('float32')

x_trainF /= 255.0                       # 0~1로 정규화
x_testF /= 255.0

x_trainF.shape, x_testF.shape

In [ ]:
y_train10 = tf.keras.utils.to_categorical(y_train,10)   # one-hot 인코딩
y_test10 = tf.keras.utils.to_categorical(y_test,10)

y_train10.shape, y_test10.shape

# k 최근접 이웃(kNN, k-Nearest Neighbor)

In [ ]:
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# 학습
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(x_trainF, y_train)

In [ ]:
# 예측
y_pred = clf.predict(x_testF.reshape(10000, 784))

In [ ]:
print(metrics.accuracy_score(y_test, y_pred))

In [ ]:
clf.predict(x_testF[0].reshape(1, 784))

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_test[i], cmap=plt.cm.binary)    # 0 ~ 24
    plt.xlabel(str(y_test[i]) + '-->' + str(clf.predict(x_testF[i].reshape(1, 784))))
plt.show()

# 서포트 벡터 머신(support vector machine)

In [ ]:
from sklearn import metrics
from sklearn.svm import SVC

In [ ]:
# 학습
clf_svm = SVC(kernel='rbf', gamma=0.3, max_iter=100, verbose=True)
clf_svm.fit(x_trainF, y_train)

In [ ]:
# 예측
y_pred_svm = clf_svm.predict(x_testF.reshape(10000, 784))
print(metrics.accuracy_score(y_test, y_pred_svm))

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_testF[i].reshape(28, 28), cmap=plt.cm.binary)
    plt.xlabel(str(y_test[i]) + '-->' + str(clf_svm.predict(x_testF[i].reshape(1, 784))))
plt.show()

### + HOG 기술자

In [ ]:
import cv2

imsize = 28 # size of image (28x28)
# HOG parameters:
winSize = (imsize, imsize) # 28, 28
blockSize = (imsize//2, imsize//2) # 14, 14
cellSize = (imsize//2, imsize//2) #14, 14
blockStride = (imsize//4, imsize//4) # 7, 7
nbins = 9
derivAperture = 1
winSigma = -1.0
histogramNormType = 0
L2HysThreshold = 0.2
gammaCorrection = 1
nlevels = 64
signedGradients = True

# define the HOG descriptor
hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins, derivAperture, winSigma,
                        histogramNormType, L2HysThreshold, gammaCorrection, nlevels, signedGradients)

# compute HOG descriptors
x_train_hog = []
for i in range(x_train.shape[0]):
    descriptor = hog.compute(x_train[i]) # compute the HOG features
    x_train_hog.append(descriptor) # append it to the train decriptors list

x_test_hog = []
for i in range(x_test.shape[0]):
    descriptor = hog.compute(x_test[i]) # compute the HOG features
    x_test_hog.append(descriptor) # append it to the test descriptors list

#train_descriptors = np.array(train_descriptors)
x_train_hog = np.resize(x_train_hog, (x_train.shape[0], 81))

#test_descriptors = np.array(test_descriptors)
x_test_hog = np.resize(x_test_hog, (x_test.shape[0], 81))

In [ ]:
# 학습 - SVM classifier
clf_svm_hog = SVC(kernel='rbf', gamma=0.3, max_iter=100, verbose=True)
clf_svm_hog.fit(x_train_hog, y_train)

In [ ]:
# 예측 - SVM classifier
y_pred_svm_hog = clf_svm_hog.predict(x_test_hog)
print(metrics.accuracy_score(y_test, y_pred_svm_hog))

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_testF[i].reshape(28, 28), cmap=plt.cm.binary)
    plt.xlabel(str(y_test[i]) + '-->' + str(clf_svm_hog.predict(x_test_hog[i].reshape(1, 81))))
plt.show()